<a href="https://colab.research.google.com/github/namanwahi/notebooks/blob/main/tfp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
% pip install shap
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
import shap
import numpy as np
from sklearn.datasets import make_regression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

In [ ]:
N_BINS = 7
INPUT_SIZE = 15

In [ ]:
X, y_c = make_regression(
    n_samples=100000,
    n_features=INPUT_SIZE,
    n_informative=6,
)


discretizer = KBinsDiscretizer(
    n_bins=N_BINS,
    encode="onehot-dense",
    strategy="quantile",
)
y = discretizer.fit_transform(y_c.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=5)

X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print("Bin edges ", discretizer.bin_edges_[0])

(99995, 15)
(99995, 7)
(5, 15)
(5, 7)
Bin edges  [-774.11892764 -191.0953275  -101.40239604  -31.74501316   32.49907102
  102.3074637   192.62118585  751.76878099]


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(INPUT_SIZE,)),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(N_BINS, activation="softmax"),
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 217       
Total params: 697
Trainable params: 697
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    x=X_train,
    y=y_train,
    epochs=10,
    batch_size=32,
)

Epoch 1/10
3125/3125 [==============================] - 4s 1ms/step - loss: 1.9132
Epoch 2/10
3125/3125 [==============================] - 3s 1ms/step - loss: 1.3084
Epoch 3/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.9831
Epoch 4/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.7800
Epoch 5/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.6337
Epoch 6/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.5211
Epoch 7/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.4366
Epoch 8/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.3726
Epoch 9/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.3233
Epoch 10/10
3125/3125 [==============================] - 3s 1ms/step - loss: 0.2864


In [ ]:
predictions = model.predict(X_test)
print(predictions.shape)

(5, 7)


NotImplementedError: ignored